# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output_data_file")
city_df

,Cities,Latitude,Longitude,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,hami,42.80,93.45,92.08,56,0,4.50,CN,1592312993
1,akhtubinsk,48.28,46.17,89.15,56,0,11.79,RU,1592312994
2,barrow,71.29,-156.79,33.80,56,1,11.41,US,1592312994
3,harare,-17.83,31.05,66.02,56,1,9.22,ZW,1592312994
4,anadyr,64.75,177.48,41.00,56,26,6.31,RU,1592312994
...,...,...,...,...,...,...,...,...,...
557,lasa,46.62,10.70,66.99,56,75,8.05,IT,1592313111
558,porto novo,6.50,2.60,82.40,56,40,10.29,BJ,1592313111
559,homer,59.64,-151.55,51.80,56,1,6.49,US,1592313111
560,pokosnoye,55.53,101.05,55.98,56,89,7.34,RU,1592313111


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)
locations = city_df[["Latitude", "Longitude"]].astype(float)

city_df = city_df.dropna()
humidity = city_df["Humidity"].astype(float)

In [15]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
newcity_df = city_df.loc[(city_df["Temp"] <80) & (city_df["Temp"] >70) & (city_df["Windspeed"] <10) 
                         & (city_df["Cloudiness"] ==0)].dropna()

newcity_df

,Cities,Latitude,Longitude,Temp,Humidity,Cloudiness,Windspeed,Country,Date
6,cape town,-33.93,18.42,71.60,56,0,3.36,ZA,1592312995
172,mlonggo,-6.53,110.70,79.79,56,0,7.23,ID,1592313028
237,huambo,-12.78,15.74,71.60,56,0,4.12,AO,1592313041
279,saldanha,-33.01,17.94,71.60,56,0,3.36,ZA,1592313049
375,matamoros,25.53,-103.25,72.00,56,0,1.01,MX,1592313073
385,tongliao,43.61,122.27,79.05,56,0,9.44,CN,1592313075
420,dafeng,23.45,108.59,79.47,56,0,9.13,CN,1592313082
451,pimenta bueno,-11.67,-61.19,78.06,56,0,2.95,BR,1592313090
514,tamiahua,21.27,-97.45,76.08,56,0,6.55,MX,1592313102


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
newcity_df["Hotel Name"] = ""
hotel_df = newcity_df
hotel_df

,Cities,Latitude,Longitude,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
6,cape town,-33.93,18.42,71.60,56,0,3.36,ZA,1592312995,
172,mlonggo,-6.53,110.70,79.79,56,0,7.23,ID,1592313028,
237,huambo,-12.78,15.74,71.60,56,0,4.12,AO,1592313041,
279,saldanha,-33.01,17.94,71.60,56,0,3.36,ZA,1592313049,
375,matamoros,25.53,-103.25,72.00,56,0,1.01,MX,1592313073,
385,tongliao,43.61,122.27,79.05,56,0,9.44,CN,1592313075,
420,dafeng,23.45,108.59,79.47,56,0,9.13,CN,1592313082,
451,pimenta bueno,-11.67,-61.19,78.06,56,0,2.95,BR,1592313090,
514,tamiahua,21.27,-97.45,76.08,56,0,6.55,MX,1592313102,


In [9]:
params = {"radius": 50000, "keyword": "hotel", "key": g_key}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_address = requests.get(base_url, params=params)

    # convert to json
    hotel_address = hotel_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    

In [10]:
hotel_df

,Cities,Latitude,Longitude,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
6,cape town,-33.93,18.42,71.60,56,0,3.36,ZA,1592312995,Atlanticview Cape Town Boutique Hotel
172,mlonggo,-6.53,110.70,79.79,56,0,7.23,ID,1592313028,Quest Hotel Simpang Lima Semarang
237,huambo,-12.78,15.74,71.60,56,0,4.12,AO,1592313041,Hotel Ekuikui I
279,saldanha,-33.01,17.94,71.60,56,0,3.36,ZA,1592313049,Crystal Lagoon Lodge
375,matamoros,25.53,-103.25,72.00,56,0,1.01,MX,1592313073,Torreon Marriott Hotel
385,tongliao,43.61,122.27,79.05,56,0,9.44,CN,1592313075,Country Garden Phoenix Hotel Tongliao
420,dafeng,23.45,108.59,79.47,56,0,9.13,CN,1592313082,Sankee Chunxia Garden Hotel
451,pimenta bueno,-11.67,-61.19,78.06,56,0,2.95,BR,1592313090,Catuai Hotel
514,tamiahua,21.27,-97.45,76.08,56,0,6.55,MX,1592313102,Isla Tajín Beach & River Resort


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """"
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Cities</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [38]:
# Convert to list
hotel_info = hotel_df["Hotel Name"].tolist()

In [40]:
# Add marker layer ontop of heat map
# Display figure

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_info])


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))